## How to add metadata to your documents and filter searches
This notebook will walk you through how to upload metadata that provides extra information about the corpus you are ingesting with nv-ingest. It will show the requirements for the metadata file and what file types are supported. Then we will go throught he process of filtering searches, in this case, on the metadata we provided.

First step is to provide imports for all the tools we will be using.

In [ ]:
from nv_ingest_client.client import Ingestor

Next we will annotate all the necessary variables to ensure our client connects to our pipeline.

In [ ]:
model_name="nvidia/llama-3.2-nv-embedqa-1b-v2"
hostname="localhost"
collection_name = "nv_ingest_collection"
sparse = True

Instantiate your ingestor object with all the stages you want in your pipeline. Ensure that you have a vdb_upload stage in place. As this is what will load your transformed elements(data) in to the vector database. 

In [ ]:
ingestor = ( 
    Ingestor(message_client_hostname=hostname)
    .files(["data/woods_frost.pdf", "data/multimodal_test.pdf"])
    .extract(
        extract_text=True,
        extract_tables=True,
        extract_charts=True,
        extract_images=True,
        text_depth="page"
    ).embed(text=True, tables=True
    ).vdb_upload(
        collection_name=collection_name, 
        milvus_uri=f"http://{hostname}:19530", 
        sparse=sparse, 
        minio_endpoint=f"{hostname}:9000", 
        dense_dim=2048
    )
)
results = ingestor.ingest_async().result()

Once you have completed the normal ingestion the collection has been loaded into your Vector Database. If you need to reindex that data for whatever reason, you can simply run the `reindex_collection` function and supply the necessary parameters. There is a full list in the docstring of the function, with many defaults already set for you. This function is desigend to be used when the results from your ingestor pipeline are no longer available, you might have ingested this information at a previous time and the ingestor results are no longer in available in memory. This function allows you to query the data from the vector database to recreate those results and send them back into a new collection or the same collection, effectively deleting the previous information stored in that collection. 

In this example we will be reindexing under the same collection name, replacing the data in the collection. You can always supply a `new_collection_name` as one of the arguments to the function so that you can save the reindex in another collection, if you need to perform any analysis between collection data. The function supplies a `write_dir` parameter which allows you to pull the data and write in to files in batches. Currently the batch_size is automatically set to the default query batch_size for the vector database. The `write_dir` option is meant to be used when the data is larger than the available resources, with this option reindexing is slower than when holding the data in host memory.  

In [ ]:
from nv_ingest_client.util.milvus import reindex_collection

reindex_collection(
    collection_name, 
    sparse=sparse
    )